In [29]:
from flask import Flask, jsonify, render_template
from flask_cors import CORS
import pandas as pd
import psycopg2
from psycopg2 import pool
import os
from dotenv import load_dotenv
from pathlib import Path
from contextlib import contextmanager

In [30]:
# Load environment variables
load_dotenv()

True

In [31]:
app = Flask(__name__)
CORS(app)

In [32]:
# Connection pool
db_pool = None

In [33]:
def init_db_pool():
    global db_pool
    if db_pool is None:
        db_pool = pool.SimpleConnectionPool(1, 10, dsn=DATABASE_URL)

def get_db_connection():
    if db_pool is None:
        init_db_pool()
    return db_pool.getconn()

@contextmanager
def get_conn_cursor():
    conn = get_db_connection()
    try:
        yield conn, conn.cursor()
    finally:
        conn.commit()
        conn.close()

In [34]:
def create_launches_table():
    create_query = """
    CREATE TABLE IF NOT EXISTS launches (
        id SERIAL PRIMARY KEY,
        mission_name TEXT,
        launch_date TEXT,
        launch_year INT,
        success BOOLEAN,
        failure_reason TEXT,
        agency TEXT,
        payload_mass_kg FLOAT,
        source_id TEXT UNIQUE,
        company TEXT,
        location TEXT,
        date DATE,
        time TIME,
        rocket TEXT,
        mission TEXT,
        rocket_status TEXT,
        price TEXT,
        mission_status TEXT
    );
    """
    with get_conn_cursor() as (_, cur):
        cur.execute(create_query)

def load_csv_to_postgres():
    print("📥 Loading CSV into PostgreSQL...")
    csv_path = Path("launch_data.csv")  # Use same directory as app.py
    if not csv_path.exists():
        print(f"❌ CSV not found at {csv_path}")
        return

In [35]:
from pathlib import Path

# Define the path to the CSV file
csv_path = Path("launch_data.csv")

# Read the CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\Antho\OneDrive\Desktop\PROJECT 3\launch_data.csv")

In [36]:
# Rename columns to match DB
df.rename(columns={
        "Mission": "mission_name",
        "Date": "launch_date",
        "Company": "agency",
        "Rocket": "rocket",
        "RocketStatus": "rocket_status",
        "Location": "location",
        "MissionStatus": "mission_status",
        "Price": "price"
    }, inplace=True)

    # Derive extra fields
df["launch_year"] = pd.to_datetime(df["launch_date"], errors="coerce").dt.year
df["success"] = df["mission_status"].str.lower().str.contains("success", na=False)
df["failure_reason"] = df["mission_status"].where(~df["success"], None)
df["payload_mass_kg"] = 1000
df["source_id"] = df["mission_name"].fillna("Unknown") + "_" + df["launch_date"].fillna("")

    # Fill missing optional fields if needed
for col in ["company", "date", "time"]:
        if col not in df.columns:
            df[col] = None


In [28]:
@app.route("/")
def dashboard():
    return render_template("index.html")

@app.route("/api/launches")
def api_get_launches():
    with get_conn_cursor() as (_, cur):
        cur.execute("SELECT * FROM launches;")
        columns = [desc[0] for desc in cur.description]
        data = [dict(zip(columns, row)) for row in cur.fetchall()]
    return jsonify(data)

In [ ]:
def initialize_app():
    create_launches_table()
    load_csv_to_postgres()

In [ ]:
if __name__ == "__main__":
    initialize_app()
    app.run(debug=True)
